ALERTA RIO 

In [9]:
ls

data/                       downscaling_analysis.ipynb
downscaling.ipynb           graphs/


In [10]:
import json
import re
import warnings
from datetime import datetime, timedelta
from pathlib import Path
import os
import pandas as pd
import pandera as pa
from sklearn.impute import KNNImputer
from tqdm.notebook import tqdm
import import_ipynb
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from datetime import datetime, timedelta
from downscaling import fit_bernoulli_gamma
from scipy.stats import gamma
import numpy as np

In [11]:
class AlertaRio():
    
    columns_v1 = {
            "string": "Dia         Hora      HBV   15 min   01 h   04 h   24 h   96 h",
            "names": ["data", "hora", "HBV", "precipitation", "h01", "04h", "24h", "96h"],
        }
    columns_v2 = {
            "string": "Dia         Hora      HBV   05 min   10 min   15 min   01 h   04 h   24 h   96 h",
            "names": [
                "data",
                "hora",
                "HBV",
                "precipitation",
                "10min",
                "15min",
                "h01",
                "04h",
                "24h",
                "96h",
            ],
        }
    
    def __init__(self,path):
        self.data = pd.DataFrame(columns= ["datetime", "precipitation", "h01", "station"])
        self.rain_gauge_path = Path(path)
        self.stations = []
        
    def get_columns(self, filename):
    
        with open(filename) as f:
            data = f.read().splitlines()
        line = data[4].strip()
        if not len(data) > 4:
            return None
        if line == self.columns_v1["string"]:
            
            return self.columns_v1["names"]
        elif line == self.columns_v2["string"]:
            return self.columns_v2["names"]
        return None
     
    def process_station(self, station: str) -> pd.DataFrame:
        
        station_dfs = []
        months = pd.date_range(pd.Timestamp("2011-01-01"), pd.Timestamp("2024-10-01"), freq="MS")
        for month in tqdm(months, desc=f"Processing station {station}", unit="month", leave=False):
            current_year = month.year
            current_month = month.month
            
            try:
                file_name = f"{station}_{current_year:04d}{current_month:02d}_Plv.txt"
                file_path = self.rain_gauge_path/ file_name
                if file_path.exists():
                    # raise FileNotFoundError(f"File {file_path} not found")
                    names = self.get_columns(file_path)
                    if not names:
                        print(f"Columns cannot be inferred from file {file_path}.")
                    
                    df = pd.read_csv(file_path , sep=r"\s+", skiprows=5, header=None, names=names)
                    rows_to_shift = df[df["HBV"] != "HBV"].index
                    df.loc[rows_to_shift, "HBV":] = df.loc[rows_to_shift, "HBV":].shift(1, axis=1)
                    df = df[["data", "hora", "precipitation", "h01"]]
                
                    df["datetime"] = df["data"] + " " + df["hora"]
                    df["datetime"] = pd.to_datetime(
                        df["datetime"], dayfirst=True, errors="coerce", format="%d/%m/%Y %H:%M:%S"
                    )
                    df["precipitation"] = pd.to_numeric(df["precipitation"], errors="coerce")
                    df["h01"] = pd.to_numeric(df["h01"], errors="coerce")
                    df["station"] = station
                    df = df.drop(columns=["data", "hora"])
                    station_dfs.append(df)
                else:
                   print(f"File {file_path} not found")
            except Exception as e:
                print(f"Error processing station {station} at {current_year}-{current_month}: {e}")
                raise e
        #assert len(station_dfs) == len(months)
        df = pd.concat(station_dfs).sort_values(by="datetime").reset_index(drop=True)
        # df = self._impute_missing_values(df, AlertarioSchema.precipitation)
        # df = self._impute_missing_values(df, AlertarioSchema.h01)
        return df
        
    
    def list_rain_gauge_stations(self) -> list[str]:
        unique_names = set()
        pattern = re.compile(r"^(.*?)_\d{6}_Plv\.txt$")
        for file in self.rain_gauge_path.iterdir():
            filename = file.name
            match = pattern.match(filename)
            if not match:
                raise ValueError(f"Filename {filename} does not match the pattern")
            unique_names.add(match.group(1))
        return sorted(unique_names)
        
    def retrieve_all_stations_data(self):
        self.stations = self.list_rain_gauge_stations()
        for station in self.stations:
            df = self.process_station(station)
            self.data = pd.concat([self.data, df], ignore_index=True)


In [30]:
def plot_fitted_curve_and_histogram(data):
    
    stations = data['station'].unique()
    df_filtered = data[data['precipitation'] > 0]
    
    for station in stations:
        station_data = data[data['station'] == station]
        
        p, shape, loc, scale = fit_bernoulli_gamma(station_data['precipitation'])
        
        plt.figure(figsize=(10, 6))
        plt.hist(df_filtered['precipitation'], density=True, label='Data', color='blue')

        x = np.linspace(min(df_filtered['precipitation']), max(df_filtered['precipitation']), 50)
        pdf_fitted = gamma.pdf(x, a=shape, loc=0.0, scale=scale)
        plt.plot(x, pdf_fitted, 'r-', lw=2, label=f'Gamma Fit\nshape={shape:.2f}, scale={scale:.2f}')

        plt.xlabel("Precipitation")
        plt.ylabel("Fraction (Density)")
        plt.title(f'Scatter Plot with Fitted Curve - {station} station')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'histogram_fitted_gamma_curve/{station}.png')
        plt.close()

In [ ]:
alertaRio = AlertaRio("data/alertario-from-source")
alertaRio.retrieve_all_stations_data()

In [31]:
plot_fitted_curve_and_histogram(alertaRio.data)